# Week 1: Introduction to Computer Vision

## Notebook 4: Object Detection with a Pre-Trained Faster R-CNN using Torchvision

Welcome to the sixth notebook of this week's Applied AI Study Group! We will study object detection problem with Common Object Common Context (COCO) dataset provided by Microsoft. The aim of our task will be to detect the classes and locations of the objects in the given images.

This notebooks is prepared using the following two sources: [Learn OpenCV](https://www.learnopencv.com/faster-r-cnn-object-detection-with-pytorch/) and [PyTorch Official Tutorial](https://pytorch.org/docs/stable/torchvision/models.html).

### 1. Object Detection

Object detection is one of the fundamental tasks in computer vision where many applications are built on it. Object detection has a long history throughout the first years of computer vision research. We will cover a recent model applied in today's world in this notebook. We suggest that people should also study the pre-deep learning era of object detection to understand today's state-of-the-art (SOTA) better. We came a long way from sliding window approach with hand-crafted kernels or histogram of oriented gradients to fully convolutional neural networks, even more complex architectures today.

![Example Object Detection](./images/object_detection.png "Example Object Detection")

We will focus on object detection with fully convolutional neural networks in this notebook. Object detection is the task of finding all the interesting objects in an image. We generally represent objects by bounding boxes. Also, we classify each bounding box to understand what objects are present in the images. It is not an easy task since there are many objects with many categories in world. Also, even the same objects belonging to same classes are different. Objects have unknown positions, scale and aspect ratios. Also, objects are present with various conditions such as partial occlusions and illumination changes in images.

We can divide the SOTA models in two categories: one-stage and two-stage object detection models. The diference between one-stage detectors and two-stage detectors is that one-stage detectors directly regress the bounding boxes and object classes from the input image. The main advantage is they are incredibly fast. On the other hand, two-stage detectors divides the detection process into two steps. First, they utilize a feature extraction network to understand interesting regions of the given images. Then, they regress the bounding boxes and objects classes from these interesting regions. They are slow compared to one-stage detectors but they have better mean Average Precision scores than one-stage detectors. Both approaches have their advantages and disadvantages but we are not going into the details of them in this notebook.

We will use a two-stage detector called 'Faster R-CNN with ResNet50 as feature extraction network combined with Feature Pyramid Network' in this notebook.

### 2. Faster R-CNN

[Faster R-CNN](https://arxiv.org/pdf/1506.01497.pdf) is one of the crucial models proposed in object detection history. It created a breakthrough from two-stage detectors to one-stage detectors. It also provided a way to train our models end-to-end. Pushing the field one step further, they proposed Region Proposal Network (RPN) for interesting region finding in the given images. RPN extracts a set of anchors with classification (just doing: object/no-object) and bounding box regression. Then, second stage of Faster R-CNN regress the classes (e.g. human, dog, cat, etc) and the refined bounding box positions given the initial predictions.

As you can see in the figure below, Region Proposal Network finds interesting regions in the given image using the features extracted from the image. Then, we apply Region of Interest (RoI) pooling, hence, we will have these interesting regions in the extracted feature map. Then, we will regress the classes and the refined bounding box positions in the second stage.

<!-- ![Faster R-CNN Model Architecture](./images/faster_rcnn.png "Faster R-CNN") -->
<div>
    <img src="./images/faster_rcnn.png" width="500"/>
</div>

### 3. Torchvision

Torchvision package is built on PyTorch. It aims to provide popular datasets, model (including pretrained models) architectures, and image transformation techniques applied in the field of computer vision.

The installation guide and further information about Torchvision can be found in [Week 1 - Notebook 5](https://github.com/inzva/Applied-AI-Study-Group/blob/add-cv-week1/Applied%20AI%20Study%20Group%20%236%20-%20January%202022/Week%201/5-segmentation_using_pre-trained_model_with_torchvision_final.ipynb).

### 4. Imports and Checks

You should have installed Numpy and Matplotlib using `pip` and, PyTorch using [Week 0 - Notebook 2](https://github.com/inzva/Applied-AI-Study-Group/blob/add-frameworks-week/Applied%20AI%20Study%20Group%20%236%20-%20January%202022/Week%200/2-mnist_classification_convnet_pytorch.ipynb).

Opencv can be installed via:
    
    !pip install opencv-python

PIL can be installed via:

    !pip install Pillow

If the following cell runs successfully, then, you are good to go!

In [ ]:
import torchvision
import torchvision.transforms as T
import cv2
from PIL import Image
import matplotlib.pyplot as plt

We will need the category names for visualization purposes. So, we define the categories in COCO dataset as a list.

In [ ]:
# categories of COCO dataset.
COCO_INSTANCE_CATEGORY_NAMES = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella', 'N/A', 'N/A',
    'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
    'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
    'bottle', 'N/A', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl',
    'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table',
    'N/A', 'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
    'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A', 'book',
    'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]

Using torchvision package, we load the Faster R-CNN with ResNet50 as feature extraction network combined with Feature Pyramid Network model. 

If you are loading this cell for the first time, it will load the model before continuing on. If you do not clear the cache, the code will directly use the saved model in future cases.

We set our model in evaluation mode since we just want to observe what object detection looks like in this notebook.

In [ ]:
# get pre-trained model from torchvision, and put in eval mode!
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
model.eval()
# model.train() for switching to training mode!

In the following cell, you will understand how an iteration on given data can be performed for a model and label the output data for visualization purposes.

First, we load the input image using Image package from Pillow.

Second, we introduce PyTorch transform as a sequential list of transforms with a list and apply it to input data. This is just a basic tensor transform operation but you can experiment further data transform techniques by just adding them in the T.Compose list.

Third, we make forward pass using our model to obtain predictions.

Fourth, we separate the prediction results into their classes, scores, and bounding boxes as separate lists.

Then, we filter out some of the predictions based on our score threshold to reduce the number of predictions. Because, we can observe that some of the predictions corresponds to same ground-truth, so, we do not need multiple of them. You can play around the threshold while calling the function to observe changes.

Finally, we return the bounding boxes and the classes for visualization in the final step of this notebook.

In [ ]:
def get_prediction(img_path, threshold):
  img = Image.open(img_path) # Load the image
  transform = T.Compose([T.ToTensor()]) # Define PyTorch Transform
  img = transform(img) # Apply the transform to the image
  pred = model([img]) # Pass the image to the model
  pred_class = [COCO_INSTANCE_CATEGORY_NAMES[i] for i in list(pred[0]['labels'].numpy())] # Get the Prediction Score
  pred_boxes = [[(i[0], i[1]), (i[2], i[3])] for i in list(pred[0]['boxes'].detach().numpy())] # Bounding boxes
  pred_score = list(pred[0]['scores'].detach().numpy())
  pred_t = [pred_score.index(x) for x in pred_score if x > threshold][-1] # Get list of index with score greater than threshold.
  pred_boxes = pred_boxes[:pred_t+1]
  pred_class = pred_class[:pred_t+1]
  return pred_boxes, pred_class

In this cell, we call our previous function to get output vectors. Then, using Opencv, we visualize the outputs by drawing the bounding boxes with the classes on the input image.

In [ ]:
def object_detection_api(img_path, threshold=0.5, rect_th=3, text_size=3, text_th=3):

  boxes, pred_cls = get_prediction(img_path, threshold) # Get predictions
  img = cv2.imread(img_path) # Read image with cv2
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # Convert to RGB
  for i in range(len(boxes)):
    cv2.rectangle(img, boxes[i][0], boxes[i][1],color=(0, 255, 0), thickness=rect_th) # Draw Rectangle with the coordinates
    cv2.putText(img,pred_cls[i], boxes[i][0],  cv2.FONT_HERSHEY_SIMPLEX, text_size, (0,255,0),thickness=text_th) # Write the prediction class
  plt.figure(figsize=(20,30)) # display the output image
  plt.imshow(img)
  plt.xticks([])
  plt.yticks([])
  plt.show()

We call the object detection API we built throughout this notebook to observe results.

In [ ]:
# call detection function
object_detection_api('./datasets/people.jpg', threshold=0.8)

Congratulations! You have finished the first week of our program, Applied AI!